---

# **title: '머신러닝4 로지스틱 회귀'**

# **date: '2022-07-01 11:00'**

---

## 로지스틱 회귀
- 선형회귀에서 출발
- 이진 분류 문제 해결
- 클래스 확률 예측
- 딥러닝에서도 사용됨 

- P177
 + X가 사격형일 확율 30%
 + X가 삼각형일 확률 50%
 + X가 원일 확률 20%


## 데이터 불러오기
- Species(종속변수 = Y)
- Weight,Length, Diagonal,Height,Width(독립변수들)


In [48]:
import pandas as pd

fish = pd.read_csv('https://bit.ly/fish_csv_data')
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


## 데이터 탐색


In [65]:
# 종속변수
print(pd.unique(fish['Species']))#유니크함수로 스피시스열의 고유값 추출
print("")
print(fish['Species'].value_counts())

['Bream' 'Roach' 'Whitefish' 'Parkki' 'Perch' 'Pike' 'Smelt']

Perch        56
Bream        35
Roach        20
Pike         17
Smelt        14
Parkki       11
Whitefish     6
Name: Species, dtype: int64


## 데이터 가공 

In [63]:
# 판다스 데이터 프레임에서 넘파이 배열로 변환
fish_input = fish[['Weight','Length','Diagonal','Height','Width']].to_numpy() 
#  fish 데이터 프레임에서 여러열을 선택해 새로운 데이터 프레임을 넘파이 배열로 바꾸어 저장

print(fish_input.shape)

(159, 5)


In [31]:
print(fish_input[:5])

[[242.      25.4     30.      11.52     4.02  ]
 [290.      26.3     31.2     12.48     4.3056]
 [340.      26.5     31.1     12.3778   4.6961]
 [363.      29.      33.5     12.73     4.4555]
 [430.      29.      34.      12.444    5.134 ]]


- 타킷데이터, 종속변수, Y

In [32]:
fish_target = fish['Species'].to_numpy()
print(fish_target.shape)
print(fish_target[:5])

(159,)
['Bream' 'Bream' 'Bream' 'Bream' 'Bream']


## 데이터 분리
- 훈련 데이터 테스트 데이터 분리 

In [50]:
from sklearn.model_selection import train_test_split
#임의 샘플링

train_input, test_input,train_target,test_target = train_test_split(
    fish_input, fish_target, random_state= 42
)
print(train_input.shape) # 훈련데이터 값
#층화 샘플링


(119, 5)


## 표준화 전처리 
- 여기에서도 훈련 세트의 통계 값으로 테스트 세트를 변환해야 한다는 점을 잊지 마세요!!(중요)
- 훈련 세트의 평균값과 테스트 세트의 평균값는 다르다. 따라서 테스트 세트의 평균값(통계값)을 훈련세트의 평균값(통계값)으로 대체해줘야 한다.
- 데이터 가공
 + 숫자 결측치가 존재, 평균값으로 대체
 + 원본 데이터 평균으로 대치하면 안됨
 + 훈련 데이터와 테스트 데이터 분리
- 데이터 누수(Data Leakage)
 + 훈련데이터 평균값 70을 대치(기준)
 + 테스트 데이터 평균값(75)과 모든 데이터 평균값(72.5)은 기준이 안됨 
 + 참조: https://scikit-learn.org/stable/common_pitfalls.html

cf) 기준을 맞춰라 -->데이터 표준화(표준점수)
  - p97~100는 수동으로 mean,std 을 
  -'# train_scaled = (train_input - mean)/ std 
   라는 수식을 만들어 사용했으나 
  - StandardScaler 라는 매소드가 있으니 이를 이용하면 된다. 

In [34]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
#ss.fit(test_input)을 하면 안됨!!-> 훈련테스트 통계값으로 통일
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

## 모형 만들기
- K-최근접 이웃


In [36]:
from sklearn.neighbors import KNeighborsClassifier
kn = KNeighborsClassifier(n_neighbors = 3)
kn.fit(train_scaled, train_target)

print(kn.score(train_scaled, train_target))
print(kn.score(test_scaled, test_target))

0.8907563025210085
0.85


- 타깃값 확인
- 알파벳 순으로 정렬

In [37]:
print(kn.classes_)

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


- 다중분율


- 5개 샘플에 대한 예측은 어떤 확률이냐?

In [41]:
import numpy as np
proba = kn.predict_proba(test_scaled[:5])
print(kn.classes_)
print(np.round(proba,decimals= 4))


['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']
[[0.     0.     1.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     1.     0.    ]
 [0.     0.     0.     1.     0.     0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]]


- 첫번째 클래스는 Perch
 + 100% 확률로 Perch로 예측
- 네번째 클래스는 Perch
 + 66.7%확률로 Perch로 예측
 + 33.3%확률로 Roach로 예측

## 회귀식
- y= ax + b
- 양변에 로그를 취함
- 원래 값으로 돌리기 위해 양변을 다시 지수로 변환->로지스틱 회귀 

## 로지스틱 회귀로 이진분류 수행


In [42]:
char_arr = np.array(['A','B','C','D','E'])
print(char_arr[[True,False,True,False,False]])

['A' 'C']


 + 도미와 빙어의 행만 골라냄 (bream,smelt)


In [53]:
bream_smelt_indexes =(train_target == 'Bream') | (train_target == 'Smelt')
print(bream_smelt_indexes)
train_bream_smelt = train_scaled[bream_smelt_indexes]
target_bream_smelt = train_target[bream_smelt_indexes]

print(train_scaled.shape, train_bream_smelt.shape)

[ True False  True False False False False  True False False False  True
 False False False  True  True False False  True False  True False False
 False  True False False  True False False False False  True False False
  True  True False False False False False  True False False False False
 False  True False  True False False  True False False False  True False
 False False False False False  True False  True False False False False
 False False False False False  True False  True False False  True  True
 False False False  True False False False False False  True False False
 False  True False  True False False  True  True False False False False
 False False False False  True  True False False  True False False]
(119, 5) (33, 5)


- 총 119마리에서 참인 값은 33마리만 추출

## 모델 만들기

In [54]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(train_bream_smelt,target_bream_smelt)

LogisticRegression()

In [56]:
print(lr.predict(train_bream_smelt[:5]))#훈련한 모델로 5개 샘플 예측

['Bream' 'Smelt' 'Bream' 'Bream' 'Bream']


In [57]:
print(lr.predict_proba(train_bream_smelt[:5]))# 예측 확율을 출력 두번째만 도미가 아님

[[0.99759855 0.00240145]
 [0.02735183 0.97264817]
 [0.99486072 0.00513928]
 [0.98584202 0.01415798]
 [0.99767269 0.00232731]]


In [58]:
print(lr.classes_) # 음성클라스 도미(0): 양성크라스 빙어(1)

['Bream' 'Smelt']


- cf. 분류기준 : threshold 임계값 설정(경계선 설정)
 + 도미 Vs 빙어 
    - [0.51,0.49]-> 이런값은 도미인가 빙어인가?
    - [0.90,0.10]

- 계수와 절편 

In [59]:
print(lr.coef_, lr.intercept_)#로지스틱 회귀는 선형회귀와 비슷

[[-0.4037798  -0.57620209 -0.66280298 -1.01290277 -0.73168947]] [-2.16155132]


In [94]:
decisions = lr.decision_function(train_bream_smelt[:5])#decision_function()메서드로 Z값 출력
print(decisions)

[[ 13.07724442   5.67940163  -3.35341274  -3.31343798   2.17367082
  -20.94258142   6.67911528]
 [-11.87101288   2.30253045   5.38260123  -3.16152122   3.19003127
    8.30344773  -4.14607657]
 [ 12.33862012   5.65079591  -4.66939988  -2.1462105    1.70362799
  -17.38222731   4.50479367]
 [ 10.54150945   6.10969846  -4.81186721  -2.96238906   2.29032761
  -14.96402558   3.79674632]
 [ 13.67852112   5.73152066  -4.25491239  -2.55085968   1.73528849
  -20.24827704   5.90871883]]


- z값을 확율값으로 변환시켜야 함. 지수변환(p188)시켜야 함
- expit() 

In [61]:
from scipy.special import expit
print(expit(decisions))

[0.00240145 0.97264817 0.00513928 0.01415798 0.00232731]


## 다중 분류 수행하기
 -  2진분류의 확장판
 

In [68]:
# 하이퍼 파라메터 세팅
# 모형을 튜닝(잘모르면 건들지 않는게 좋음, defult값 사용)
# 모형 결과의 과대적합 또는 과소적합을 방지하기 위한 것
lr = LogisticRegression(C =20 , max_iter = 1000)
lr.fit(train_scaled, train_target)
print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled, test_target))

0.9327731092436975
0.925


In [73]:
print(lr.predict(test_scaled[:5]))

['Perch' 'Smelt' 'Pike' 'Roach' 'Perch']


In [70]:
proba = lr.predict_proba(test_scaled[:5])
print(np.round(proba, decimals = 3))
print(lr.classes_)

[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]
['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


- 다중 분류일 경우 선형 방정식은 어떤 모습일까?
- 분류 7개 컬럼 값 5개 


In [83]:
print(lr.coef_,lr.intercept_)
print("")
print(lr.coef_.shape, lr.intercept_.shape)

[[-1.49002087 -1.02912886  2.59345551  7.70357682 -1.2007011 ]
 [ 0.19618235 -2.01068181 -3.77976834  6.50491489 -1.99482722]
 [ 3.56279745  6.34357182 -8.48971143 -5.75757348  3.79307308]
 [-0.10458098  3.60319431  3.93067812 -3.61736674 -1.75069691]
 [-1.40061442 -6.07503434  5.25969314 -0.87220069  1.86043659]
 [-1.38526214  1.49214574  1.39226167 -5.67734118 -4.40097523]
 [ 0.62149861 -2.32406685 -0.90660867  1.71599038  3.6936908 ]] [-0.09205179 -0.26290885  3.25101327 -0.14742956  2.65498283 -6.78782948
  1.38422358]

(7, 5) (7,)


## 평가지표
- 회귀 평가지표
  -> 결정계수($R^2$)P.121
  + $1-[(타깃-예측)^2의 합/(타깃-평균)^2합]$
- MAE, MSE, RMSE
  + (실제 - 예측) =오차
  + MAE(mean absolute errer): 오차의 절댓값의 평균
  + MSE(m Squared e): 오차의 제곱의 평균
  + RMSE(Root MSE): MSE에 제곱근을 취한값

- 좋은 모델이란 
 + 결정계수 :1에 수렴하면 좋은 모델
 + MAE외 :0에 수렴하면 좋은 모델

In [90]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

true = np.array([1,2,3,2,3,5,4,6,5,6,7,8,8]) #실제값
preds = np.array([1,1,2,2,3,4,4,5,5,7,7,6,8])#예측값

#절대값 오차의 평균
mae = mean_absolute_error(true, preds)
print("mae=",mae)
#제곱 오차의 평균
mse = mean_absolute_error(true, preds)
print("mse=",mse)
#mse제곱근
rmse =np.sqrt(mse)
print("rmse=",rmse)
#결정계수
r2 = r2_score(true, preds)
print("r2=",r2)

mae= 0.5384615384615384
mse= 0.5384615384615384
rmse= 0.7337993857053428
r2= 0.8617021276595744


## 분류 오차 행렬
- 오차 행렬
- 실제 값
 + [빙어, 도미, 도미, 빙어, 도미]
- 예측 값
 + [빙어, 빙어, 도미, 빙어, 빙어]

   - TP(빙어를 빙어로 예측):2
   - TN(도미를 도미로 예측):1
   - FP(실제도미,예측 빙어):2
   - FN(실제빙어,예측 도미):0
- 모형의 정확도 3/5 =60% 
- 사이킷런에 분류오차행렬 함수가 있다.
- TP,TN,FP,FN(5,4,3,7)
 + 정확도(5+4/5+5+3+7)
 + 정밀도(precision:5/5+3):양성이라 예측(TP+FP)중 실제 양성값(TP)의 비율(스팸메일)->실수를 옳다고 생각하면 안되는 값
 + 재현율(5/5+7):실제 양성(TP+FN) 값 중 양성으로 예측한 값(TP)의 비율 (암진단)->사실을 거짓으로 판단하면 큰일나는 값
 + 로그손실
 + ROC Curve(=AUC)
- 코로나 검사
 + 양성(1) : 음성(99)
 + 머신러닝 모형 :98%/ 정밀도 99
 + 인간 음성진단 :99%/ 정밀도 95
 + 검사자가 실제는 양성이나 진단은 음성으로 내릴 가능성이 높음(의료사고)-재현율로 파악하는 것이 옳다.

In [92]:
from sklearn.metrics import confusion_matrix

true = [0,1,1,0,0]
preds = [1,0,0,0,0]

confusion_matrix(true, preds)

array([[2, 1],
       [2, 0]])